# Lesson4 ニューラル翻訳モデルを作ってみる(Seq2Seq, Attention)

## Section1 解説

今回は、深層学習を使用した翻訳モデルを作成する。

まず、機械学習における言語の扱いとして言語モデルに触れてから、各単語の扱い方を学ぶ。

そのあと、翻訳モデルとして深層学習で頻繁に使用される系列変換モデルについて解説する。

### 1.1 言語モデル

言語モデルとは、ある文章が生成される過程を確率的にモデル化したもの。

言語モデルを獲得することで、文章のもっともらしさを測ったり、またもっともらしい文章を生成したりすることが可能となる。

具体的にいうと、文章を単語ごとに分割して並べ、その単語列が生成される確率をモデル化する。

各単語の生成は、周囲の単語による条件付き確率分布によって決まるといえる。(ある単語の
  出現確率は周囲の単語に依存すると考えられるので)

例えば、
- p(晴れ | 今日の天気は＿です。) : 「今日の天気は＿です。」の空欄の単語が「晴れ」である確率
- p(晴れ | あの＿の名前はジョンです。) : 「あの＿の名前はジョンです。」の空欄の単語が「晴れ」である確率

は、同じ「晴れ」の出現確率でも全く異なる値を取ると予想でき、確かに周囲の単語に依存した条件付き確率を考える必要があると分かる。

したがって、周囲の単語による条件付き確率分布を各条件下でモデル化していくことが、文章全体の生成モデルを考えることに相当する。

こうした条件付き確率分布を求めることができれば、逐次的に単語の生成を行うことで、もっともらしい文章の生成が可能となる。

このモデル化においてニューラルネットワークを用いたものを**ニューラル言語モデル**とよび、深層学習の発展とともに注目されている。

ニューラル言語モデルは大きく分けて次の2つがある。
- **順伝播型ニューラル言語モデル**(**FFNN言語モデル**) : t番目の単語予想に直前の数単語分(固定長)だけ用いるモデルで、Denseレイヤーを用いる。
- **再帰型ニューラル言語モデル**(**RNN言語モデル**) : t番目の単語予想にそれまでの系列全てを用いるモデルで、RNNやLSTMなどが相当する。

ここでは、可変長入力に対応しやすく、系列データにふさわしいRNN言語モデルに注目していく。

### 1.2 単語のベクトル化と分散表現
言語処理では、単語を扱うが、そのままでは数学的・計算的に扱いにくいので、**数値化**や**ベクトル化**をする。

**数値化**

単純な数値化として出現順や頻度順での番号割り振りが考えられるが、kerasではkeras.preprocessing.text.Tokenizerクラスで簡単に実行できる。

主な引数
- num_words : 利用する単語の最大数(指定するデータセット中の頻度上位num_wordsの単語のみ使用)
- filters : 句読点などをフィルタする文字のリスト
- lower : テキストを小文字に強制するか
- spilit : 単語を分割するセパレータ
- char_level : 文字ごとに分割・数値化するか

```python
keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                    lower=True, split=" ", char_level=False)
```

主なメソッド
- fit_on_texts(texs) : 入力＝学習に使う文章のリスト、出力＝なし
- texts_to_sequence(texts) : 入力＝数値化する文章のリスト、出力＝数値化された文章のリスト

**ベクトル化**

ベクトル化には様々な手法が存在する。もっとも単純なのはone_hot表現に変換するもの。しかし、one_hot表現に変換する方法では、単語同士の意味の近さが含まれない不便な表現となってしまう。

**分散表現**は単語同士の意味の近さを機械学習でモデル化したうえで、単語をベクトル化するもの

分散表現の比較的単純なものとして、**埋め込み**(**Embedding**)とよばれるものがある。

埋め込みでは、one_hot表現よりも小さい次元に特徴量を圧縮してベクトル化するが、この圧縮の仕方はニューラルネットワークにより学習する。

(one_hot表現では理想的にすべての単語分の次元が必要になるが、Embeddingを用いることでより手ごろな次元で扱うことができる。)

one_hot表現のデータをEmbeddingによって変換する際には、下図のような行列ベクトル積が行われる。

<img src=figures/embedding.png>

kerasではこのEmbeddingもkeras.layer.embeddings.Embeddingとして使用できる。

※one_hot表現への変換とEmbeddingによる変換は独立していることも多いが、Kerasでは統合されていて、数値化された系列をそのまま入力する。

主な引数
- input_dim : 単語数(＝入力データの最大インデックス+1)
- output_dim : 出力次元(何次元に圧縮するか)
- embedding_{initializer, regularizer, constraint} : embedding行列のInitializers, Regularizers, Constraints
- masks_zero : 入力系列中の0をパディング(系列の長さを統一するために追加される無意味な要素)と解釈し、無視するかどうか
- input_length : 入力の系列長

```python
keras.layers.embeddings.Embedding(input_dim, output_dim,
                                  embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None,
                                  embeddings_constraint=None, mask_zero=False, input_length=None)
```

他にも分散表現を実現するものとして**word2vec**(skip-gramやCBoWというモデルを総称したもの )がある。(紹介のみ)

### 1.3 系列変換モデル
言語処理のモデルとしては、IMDB(感情ラベル付き映画レビューのデータセット)を用いた文章感情の分類などといった分類タスクもある。

しかし、実際有用なのは、ある文章を受けて異なる文章を生成するようなモデルであると考えられる。

そのようなモデルを**系列変換モデル**(**Seq2Seqモデル**)という。

大まかな構成としては下図のようになり、役割の違いによって2つのモジュールに分けて考えることが可能となる。

- **符号化器**(左3ユニット) : 入力系列を受け取って抽象化する。
- **復号化器**(右5ユニット) : 抽象化された入力系列を加味しつつ、真の出力系列を元に各々1つ先の単語を出力する

この符号化器と復号化器という2つのモジュールからなっているという側面に注目してSeq2Seqモデルをencorder-decorderモデルとよぶこともある。

<img src=figures/seq2seq.png>

実際にSeq2Seqモデルを構成する場合には大まかに以下のようなレイヤー構成にする。

1. **符号化器Embeddingレイヤー** : 特徴量変換(入力系列のone_hot表現⇒埋め込み表現)
1. **符号化器再帰レイヤー** : 入力系列を"抽象化"(最終的な隠れ状態ベクトルの取得が目的、符号化器の途中の出力系列には興味がない)
1. **復号化器Embeddingレイヤー** : 特徴量変換((復号化器出力レイヤーで生成された)直前の出力単語のone_hot表現→埋め込み表現)
1. **符号化器再帰レイヤー** : 抽象化した入力系列を加味しながら(状態ベクトルの初期化値として使う)、現在の単語の1つ先の単語を出力
1. **復号化器出力レイヤー** : 復号化器再帰レイヤーの出力系列をもとにして目的の出力系列に変換する(隠れ状態ベクトル表現→one_hot表現)

つまり、RNN言語モデルで符号化器と復号化器の骨格を構成し、入力や出力との間をEmbeddingレイヤー(&Denceレイヤー)で取り持っている。

再帰レイヤーにはRNNやLSTMのほかにもGRUなどがあり、単方向か双方向か、何層積み重ねるかなど幅広い選択肢があり、工夫が求められる。

このSeq2Seqモデルを用いれば、機械翻訳や文書要約、質問応答などの文章入力に対して文章出力が求められるタスクに生かすことができる。

Seq2Seqモデルの作成にあたっては、再帰レイヤーで　①隠れ状態ベクトルが取得できる　②出力系列の取得ができる　必要がある。

具体的には、
- ①隠れ状態ベクトル(LSTMの$c_t, h_t$に相当)を取得 : 引数にreturn_state=Trueを指定
- ②出力系列を取得 : 引数にreturn_sequences=Trueを指定

とすればよく、LSTMレイヤーを生成する際の引数として指定する。

実際のコード例はSection2の実装コードを参照。

LSTMに関してはLesson3を参照。

### 1.4 Functional API
実はSeq2SeqモデルはこれまでのSequentialクラスによるモデル構築は出来ない。

Sequentialクラスを用いる場合はadd関数を使用して簡単にモデル構築が可能であるが、途中に分岐や合流があるような複雑なモデルは作成できない。

複雑なモデルの構築方法が**Functional API**である。Seq2SeqモデルもFunctional APIによる実装が可能。

Functional APIの実装上の特徴
- **Inputレイヤー**から構築を始める
- 各レイヤ―の返り値(テンソル)を次のレイヤ―の入力として順々に構築していく
- **keras.models.Modelクラス**に入力と出力を指定することでモデルを生成する

一度Modelクラスのインスタンスをつくってしまえば、あとの学習等はSeqentialクラスによる場合と同様。

ここでは概念を知るために、普通の全結合ネットワークに
よる例を以下に示す。(Seq2Seqモデルの実装時にも後述する)


In [1]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist       # データ読み込み用
from keras.utils import to_categorical # データ読み込み用

# Inputレイヤーからスタート（返り値はテンソル）
inputs = Input(shape=(784,))

# レイヤークラスのインスタンスはテンソルを引数に取れる（返り値はテンソル）
x = Dense(128, activation='relu')(inputs)      # InputレイヤーとDenseレイヤー(1層目)を接続
x = Dense(64, activation='relu')(x)            # Denseレイヤー(1層目)とDenseレイヤー(2層目)を接続
output_layer = Dense(10, activation='softmax') # レイヤーのインスタンス化を切り分けることももちろん可能
                                               # (別のモデル構成時にこのレイヤーを指定・再利用することも可能になる)
predictions = output_layer(x)                  # Denseレイヤー(2層目)とDenseレイヤー(3層目)を接続

# Modelクラスを作成（入力テンソルと出力テンソルを指定すればよい）
model = Model(inputs=inputs, outputs=predictions) # これで、「(784,)のInputを持つDense3層」構成のモデルが指定される

# 以降はSequentialと同じ
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(x_train, y_train)

C:\Users\yanor\Anaconda3\envs\matsuo\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/1
60000/60000 [==============================] - 17s 281us/step - loss: 14.5177 - acc: 0.0993
